In [1]:
!sudo apt-get install -y build-essential libasound2-dev libjack-dev portaudio19-dev
!pip install magenta --ignore-installed llvmlite




build-essential is already the newest version (12.8ubuntu1.1).
The following additional packages will be installed:
  libjack0 libportaudio2 libportaudiocpp0 uuid-dev
Suggested packages:
  libasound2-doc jackd1 portaudio19-doc
The following packages will be REMOVED:
  libjack-jackd2-0
The following NEW packages will be installed:
  libasound2-dev libjack-dev libjack0 libportaudio2 libportaudiocpp0
  portaudio19-dev uuid-dev
0 upgraded, 7 newly installed, 1 to remove and 76 not upgraded.
Need to get 625 kB of archives.
After this operation, 3340 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 libjack0 amd64 1:0.125.0-3build2 [93.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libasound2-dev amd64 1.2.2-2.1ubuntu2.5 [104 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 uuid-dev amd64 2.34-0.1ubuntu9.3 [33.6 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal/universe amd64 libjack-dev amd64 1:0

In [2]:
import magenta
from magenta.scripts.convert_dir_to_note_sequences import convert_midi
from magenta.music.sequences_lib import split_note_sequence
import tensorflow.compat.v1 as tf
import os

prepath = "/kaggle/input/jazzmidi0-1499/midi"
all_ns = []
count = 0
for file in os.listdir(prepath):
    try:
        ns = convert_midi(prepath, "midi", os.path.join(prepath, file))

    except: 
        print("Error on:", file)
        count += 1
        continue
    
    ns_split = split_note_sequence(ns,
                            20,
                            False)
#     print(len(ns_split))
    count += 1
    all_ns += ns_split
    if count%100 == 0:
        print(count)

print(len(all_ns))
!mkdir /kaggle/tmp/
if (os.path.exists("/kaggle/tmp/jazz.tfrecord") == False):
    f = open("/kaggle/tmp/jazz.tfrecord", "w")
with tf.io.TFRecordWriter("/kaggle/tmp/jazz.tfrecord") as writer:
    for sequence in all_ns:
        writer.write(sequence.SerializeToString())


/opt/conda/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/opt/conda/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


100
200
300
400
500
600
700
800
8127


In [3]:
from note_seq.protobuf import music_pb2
from magenta.pipelines import pipeline, note_sequence_pipelines, statistics
from magenta.models import polyphony_rnn
from magenta.models.polyphony_rnn import polyphony_lib, polyphony_model, polyphony_rnn_pipeline
from magenta.pipelines import dag_pipeline
from magenta.pipelines import event_sequence_pipeline
from magenta.pipelines import pipelines_common


class PolyphonicSequenceExtractor(pipeline.Pipeline):
  """Extracts polyphonic tracks from a quantized NoteSequence."""

  def __init__(self, min_steps, max_steps, name=None):
    super(PolyphonicSequenceExtractor, self).__init__(
        input_type=music_pb2.NoteSequence,
        output_type=polyphony_lib.PolyphonicSequence,
        name=name)
    self._min_steps = min_steps
    self._max_steps = max_steps

  def transform(self, input_object):
    quantized_sequence = input_object
    poly_seqs, stats = polyphony_lib.extract_polyphonic_sequences(
        quantized_sequence,
        min_steps_discard=self._min_steps,
        max_steps_discard=self._max_steps)
    self._set_stats(stats)
    return poly_seqs

# changed from the library version: remove transposition from the pipeline
def get_pipeline_mine(config, min_steps, max_steps, eval_ratio):
  """Returns the Pipeline instance which creates the RNN dataset.
  Args:
    config: An EventSequenceRnnConfig.
    min_steps: Minimum number of steps for an extracted sequence.
    max_steps: Maximum number of steps for an extracted sequence.
    eval_ratio: Fraction of input to set aside for evaluation set.
  Returns:
    A pipeline.Pipeline instance.
  """


  partitioner = pipelines_common.RandomPartition(
      music_pb2.NoteSequence,
      ['eval_poly_tracks', 'training_poly_tracks'],
      [eval_ratio])
  dag = {partitioner: dag_pipeline.DagInput(music_pb2.NoteSequence)}

  for mode in ['eval', 'training']:
    time_change_splitter = note_sequence_pipelines.TimeChangeSplitter(
        name='TimeChangeSplitter_' + mode)
    quantizer = note_sequence_pipelines.Quantizer(
        steps_per_quarter=config.steps_per_quarter, name='Quantizer_' + mode)
    poly_extractor = PolyphonicSequenceExtractor(
        min_steps=min_steps, max_steps=max_steps, name='PolyExtractor_' + mode)
    encoder_pipeline = event_sequence_pipeline.EncoderPipeline(
        polyphony_lib.PolyphonicSequence, config.encoder_decoder,
        name='EncoderPipeline_' + mode)

    dag[time_change_splitter] = partitioner[mode + '_poly_tracks']
    dag[quantizer] = time_change_splitter
    dag[poly_extractor] = quantizer
    dag[encoder_pipeline] = poly_extractor
    dag[dag_pipeline.DagOutput(mode + '_poly_tracks')] = encoder_pipeline

  return dag_pipeline.DAGPipeline(dag)

In [4]:
pipeline_instance = get_pipeline_mine(
      min_steps=80,  # 5 measures
      max_steps=None,
      eval_ratio=0.2,
      config=polyphony_model.default_configs['polyphony'])

input_dir = '/kaggle/tmp/jazz.tfrecord'
output_dir = '/kaggle/tmp/split'
pipeline.run_pipeline_serial(
  pipeline_instance,
  pipeline.tf_record_iterator(input_dir, pipeline_instance.input_type),
  output_dir)

In [5]:
print(sum(1 for _ in tf.python_io.tf_record_iterator('/kaggle/tmp/split/eval_poly_tracks.tfrecord')),
sum(1 for _ in tf.python_io.tf_record_iterator('/kaggle/tmp/split/training_poly_tracks.tfrecord')))

1571 6156


In [6]:
#Different Learning Rates: 0.001, 0.005, 0.01, 0.02, 0.05, 0.1
!polyphony_rnn_train \
--run_dir=/kaggle/working/polyphony_rnn/logdir/run-001 \
--sequence_example_file=/kaggle/tmp/split/training_poly_tracks.tfrecord \
--num_training_steps=200 \
--hparams="learning_rate=0.001" \
--num_checkpoints=100 
# --summary_frequency=10

!polyphony_rnn_train \
--run_dir=/kaggle/tmp/polyphony_rnn/logdir/run-001 \
--sequence_example_file=/tmp/polyphony_rnn/split/eval_poly_tracks.tfrecord \
--num_eval_examples=1000 \
--eval


!polyphony_rnn_train \
--run_dir=/kaggle/working/polyphony_rnn/logdir/run-005 \
--sequence_example_file=/kaggle/tmp/split/training_poly_tracks.tfrecord \
--num_training_steps=200 \
--hparams="learning_rate=0.005" \
--num_checkpoints=100 
# --summary_frequency=10

!polyphony_rnn_train \
--run_dir=/kaggle/tmp/polyphony_rnn/logdir/run-005 \
--sequence_example_file=/tmp/polyphony_rnn/split/eval_poly_tracks.tfrecord \
--num_eval_examples=1000 \
--eval


!polyphony_rnn_train \
--run_dir=/kaggle/working/polyphony_rnn/logdir/run-01 \
--sequence_example_file=/kaggle/tmp/split/training_poly_tracks.tfrecord \
--num_training_steps=200 \
--hparams="learning_rate=0.01" \
--num_checkpoints=20

!polyphony_rnn_train \
--run_dir=/kaggle/tmp/polyphony_rnn/logdir/run-01 \
--sequence_example_file=/tmp/polyphony_rnn/split/eval_poly_tracks.tfrecord \
--num_eval_examples=1000 \
--eval


!polyphony_rnn_train \
--run_dir=/kaggle/working/polyphony_rnn/logdir/run-02 \
--sequence_example_file=/kaggle/tmp/split/training_poly_tracks.tfrecord \
--num_training_steps=200 \
--hparams="learning_rate=0.02" \
--num_checkpoints=20

!polyphony_rnn_train \
--run_dir=/kaggle/tmp/polyphony_rnn/logdir/run-02 \
--sequence_example_file=/tmp/polyphony_rnn/split/eval_poly_tracks.tfrecord \
--num_eval_examples=1000 \
--eval


!polyphony_rnn_train \
--run_dir=/kaggle/working/polyphony_rnn/logdir/run-05 \
--sequence_example_file=/kaggle/tmp/split/training_poly_tracks.tfrecord \
--num_training_steps=200 \
--hparams="learning_rate=0.05" \
--num_checkpoints=20 

!polyphony_rnn_train \
--run_dir=/kaggle/tmp/polyphony_rnn/logdir/run-05 \
--sequence_example_file=/tmp/polyphony_rnn/split/eval_poly_tracks.tfrecord \
--num_eval_examples=1000 \
--eval


!polyphony_rnn_train \
--run_dir=/kaggle/working/polyphony_rnn/logdir/run-1 \
--sequence_example_file=/kaggle/tmp/split/training_poly_tracks.tfrecord \
--num_training_steps=200 \
--hparams="learning_rate=0.1" \
--num_checkpoints=20

!polyphony_rnn_train \
--run_dir=/kaggle/tmp/polyphony_rnn/logdir/run-1 \
--sequence_example_file=/tmp/polyphony_rnn/split/eval_poly_tracks.tfrecord \
--num_eval_examples = 1000 \
--eval


!cp -R /kaggle/tmp/polyphony_rnn/logdir /kaggle/working/logdir

/opt/conda/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/opt/conda/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
I0412 14:53:38.919970 130318506518336 events_rnn_graph.py:96] hparams = {'batch_size': 64, 'rnn_layer_sizes': [256, 256, 256], 'dropout_keep_prob': 0.5, 'attn_length': 0, 'clip_norm': 5, 'learning_rate': 0.001, 'residual_co